# Python @ Esri

# Topics

- Administering your GIS
- Working with Users
- Advanced Workflows

# Connecting to your GIS

First we need to grab the `GIS` module.

In [67]:
from arcgis.gis import GIS

## Where is your GIS?

ArcGIS Online is the default GIS.

In [ ]:
from arcgis.gis import GIS

# ArcGIS Online is the default
GIS()

If you need to be more specific, use the `url` argument.

Here we'll connect to a specific **ArcGIS Online Organization**.

In [ ]:
# connect to an organization
GIS(url='https://dbsne.maps.arcgis.com')

And this time, to an **ArcGIS Enterprise**.

In [ ]:
# conect to ArcGIS Enterprise
GIS(url='https://esri.northeurope.cloudapp.azure.com/portal')

## Authentication

If no authentication scheme is used, your connection will be **anonymous**.

In [ ]:
# connect anonymously
gis = GIS()

print(gis.users.me )

The most basic way to authenticate is with a username and password.

In [ ]:
# username and password
gis = GIS(
  username='tech_trekkie',
  password='SidTheKid87'
)

gis.users.me

My favorite way to authenticate is with **Profiles** to store credentials using my operating system.

In [ ]:
# save a profile ⭐
gis = GIS(
  username='tech_trekkie',
  password='SidTheKid87',
  profile='tech_trek_profile'
)

gis.users.me

Now you can use those stored credentials! (uses [keyring](https://pypi.org/project/keyring/)).

In [ ]:
new_gis_who_dis = GIS(profile='tech_trek_profile')

new_gis_who_dis.users.me

Another super easy way to connect is by using ArcGIS Pro's stored credentials.

⚠️This requires ArcPy!

In [ ]:
# Use active Portal in ArcGIS Pro
gis = GIS('pro')

gis

# Working with Users

Let's set up our GIS connection.

In [ ]:
gis = GIS(profile='joshdbs')
gis

## Who are our Users?

Let's see a breakdown of the users in our GIS by fetching user counts by type.

❗The result is going to be a `pandas` DataFrame.

In [ ]:
# get users by user type
gis.users.counts(type='user_type')

We can leverage `pandas` to quickly get a total count of users.

In [ ]:
# using pandas dataframe
df['count'].sum()

## Creating a User

Let's first see what is needed to create a user.

In [134]:
# Open the help for the `arcgis.gis.users.create()` method
gis.users.create?

The first thing we need to know more about is the password policy. [Read the docs](https://developers.arcgis.com/rest/users-groups-and-items/update-security-policy.htm).

In [ ]:
gis.admin.password_policy.policy

Next, we need to understand the **roles** in this GIS. It's important to get this value dynamically from the GIS, as these ID's could change in future releases.

In [ ]:
roles = gis.users.roles.all()
roles

Let's create a **Data Editor**. We'll need the `role_id` in order to create this user.

In [ ]:
role_id = roles[1].role_id
role_id

Now we have what we need from the GIS – let's create a new user.

In [ ]:
user = gis.users.create(username="obi_wan_esri",
                password='Us3Th3F0rc3',
                firstname="Ben",
                lastname="Kenobi",
                email="bkenobi@esri.com",
                role=role_id, # ⚠️ Avoid hardcoding this.
                user_type="creatorUT")
user

And to prove this user exists in the GIS, let's search for them...

In [ ]:
user = gis.users.get('obi_wan_esri')
user

## Editing a User

Editing the properties of a user is super simple with `update`.

In [ ]:
user.update(first_name='Obi Wan', thumbnail='./data/obi_wan.png')

And to verify our changes...

In [ ]:
user

## Working with a User's Items

First, let's double check that this user doesn't have any items yet.

In [191]:
user.items()

[]

Let's add a dataset and publish it as a Feature Layer on behalf of the user.

In [192]:
user

<User username:obi_wan_esri>

In [193]:
gis.users.me.role

'org_admin'

In [198]:
fp = "./data/crimes2.zip"
item_properties = {
    'title' : "Crimes 3",
    'type' : 'Shapefile',
    'tags' : ['Crimes']
}
shp_item = gis.content.add(data=fp, item_properties=item_properties, owner=user)
crimes_item = shp_item.publish()
crimes_item

<Item title:"Crimes 3" type:Feature Layer Collection owner:obi_wan_esri>

In [199]:
user.items()

[<Item title:"Crimes 3" type:Shapefile owner:obi_wan_esri>,
 <Item title:"Crimes 3" type:Feature Layer Collection owner:obi_wan_esri>]

## Managing Credits

Credits only apply to ArcGIS Online Organizations

In [ ]:
gis = GIS(profile='joshdbs')
user = gis.users.get('tech_trekkie')

Total number of credits in the Organization

In [ ]:
# note that credit allocation can be disabled!
#gis.admin.credits.enable()

gis.admin.credits.credits

Default credit limit

In [ ]:
gis.admin.credits.default_limit

Available credits (for a user)

In [ ]:
user.assignedCredits

Allocate Credits

In [ ]:
gis.admin.credits.allocate(username=user.username, credits=1000)
user.availableCredits

In [ ]:
user.availableCredits

Deallocate Credits

In [ ]:
gis.admin.credits.deallocate(username=user.username)
user.assignedCredits

## Deleting a User

We can delete our user with a single method. Note that users cannot be deleted if they own content – the `reassign_to` argument makes it easy to reassign content before deleting.

In [ ]:
# Delete user, reassigning content to myself
user.delete(reassign_to=gis.users.me.username)

# Cloning Content

We need two GIS connections this time.

In [ ]:
# connect to "source" GIS
gis_source = GIS(
  profile='joshdbs',
  set_active=False
)

# connect to "destination" GIS
gis = GIS(profile='portal')

Find the item we want to clone.

In [ ]:
item = gis_source.content.search(
    query='owner: joshdbs AND title: crimes AND tags: tatooine',
    item_type='Feature Layer'
)[0]
item

Finally, we will use `clone_items`.

In [ ]:
# clone items
gis.content.clone_items(items=items_to_clone)

Let's search our destination GIS to see the cloned item.

In [ ]:
items = gis.content.search(
    query='owner: siteadmin AND title: crimes AND tags: tatooine',
    item_type='Feature Layer'
)
items[0]

# Understand your GIS

Let's connect to our GIS.

In [ ]:
gis = GIS(profile='joshdbs')
gis

## GIS History

As the admin of a GIS, we have access to the history of our GIS.

Let's use `matplotlib` and `pandas` to visualize a few things.

In [ ]:
# notebook magic
%matplotlib inline

# dependencies
import datetime
import pandas as pd

# set up a time interval
then = datetime.datetime.now() - datetime.timedelta(days=4)

# get the history for our GIS as CSV, then read that with pandas
df = pd.read_csv(gis.admin.history(start_date=then, num=100000))

# preview our dataset
df.head()

Let's see what our data looks like as a chart.

In [ ]:
df.action.value_counts().plot.bar()

## Enterprise Logs

Next, let's check out the logs from an instance of ArcGIS Enterprise.

In [ ]:
gis = GIS(profile='portal')
logs = gis.admin.logs
logs

What are our current log settings?

In [ ]:
logs.settings

We can query the logs.

In [ ]:
# dependencies
import datetime
import pandas as pd

# query logs from the last 10 days
results = logs.query(start_time=datetime.datetime.now() - datetime.timedelta(days=10))

# put the logs into a pandas dataframe
df = pd.DataFrame(results['logMessages'])

df.head()

And finally we can generate a simple chart of the severity (`type`) of our log records.

In [ ]:
# notebook magic
%matplotlib inline

# plot the dataframe
df.type.value_counts().plot.bar()